In [1]:
import pandas as pd
import numpy as np
import dash
import flask
from dash.dependencies import Input, Output
import dash_core_components as dcc
import dash_html_components as html
from pandas_datareader import data as web
from datetime import datetime as dt

In [2]:
import pymongo

In [3]:
from dotenv import load_dotenv
load_dotenv()
import os
password = os.getenv("PASSWORD")
username = os.getenv("USERNAME")

In [4]:
client = pymongo.MongoClient('mongodb://%s:%s@127.0.0.1' % (username, password))
db = client["1-grams"]

In [ ]:
import csv
wordlist = []
with open('wordlist.csv', 'rt') as f:
    reader = csv.reader(f)
    wordlist = list(reader)

In [5]:
def load(word):
    try:
        df = pd.DataFrame(list(db["tweets"].find({ "word": word})))
        df = df.sort_values(by=['time'])
        df['year'] = [date.year for date in df['time']]
        df['day'] = [date.timetuple().tm_yday for date in df['time']]
        return df
    except:
        print(str("No timeseries found for "+word))
        df = pd.DataFrame()
        return df

# Input box for query

In [ ]:
external_stylesheets = ['styles.css']

server = flask.Flask(__name__)

app = dash.Dash(__name__, server=server, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1('Twitter 1-grams'),
    dcc.Input(id='my-id', value='@realdonaldtrump', type='text'),
    dcc.Graph(id='my-graph'),
])

@app.callback(Output('my-graph', 'figure'), [Input('my-id', 'value')])
def update_graph(input_value):
    data=[]
    figure = {'data':data}
    values = input_value.split(',')
    for item in values:
        df = load(item)
        data.append({'x':df['time'], 'y':df['counts'],'name':item})
    return figure


if __name__ == '__main__':
    app.run_server()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [02/Apr/2019 15:37:53] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:37:53] "GET /styles.css HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:37:54] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:37:54] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:37:54] "GET /_favicon.ico HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:37:55] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:37:59] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:38:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:38:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:38:01] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:38:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:38:02] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0

No timeseries found for 


127.0.0.1 - - [02/Apr/2019 15:38:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:38:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:38:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:38:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:38:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:38:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:38:11] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:38:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:38:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:38:12] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:38:13] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:38:14] "POST /_dash-update-component HTTP/1.1" 200 -
127.

No timeseries found for trunip


...

127.0.0.1 - - [02/Apr/2019 15:52:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:52:56] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:52:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:52:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:52:57] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:52:57] "POST /_dash-update-component HTTP/1.1" 200 -
[2019-04-02 15:52:57,724] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/janeadams/.conda/envs/test_env/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 2656, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 108, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 132, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", l

No timeseries found for 


127.0.0.1 - - [02/Apr/2019 15:53:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:53:08] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:53:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:53:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:53:09] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:53:09] "POST /_dash-update-component HTTP/1.1" 200 -
[2019-04-02 15:53:10,030] ERROR in app: Exception on /_dash-update-component [POST]
Traceback (most recent call last):
  File "/home/janeadams/.conda/envs/test_env/lib/python3.7/site-packages/pandas/core/indexes/base.py", line 2656, in get_loc
    return self._engine.get_loc(key)
  File "pandas/_libs/index.pyx", line 108, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/index.pyx", line 132, in pandas._libs.index.IndexEngine.get_loc
  File "pandas/_libs/hashtable_class_helper.pxi", l

No timeseries found for 


127.0.0.1 - - [02/Apr/2019 15:53:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:53:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:53:23] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [02/Apr/2019 15:53:23] "POST /_dash-update-component HTTP/1.1" 200 -


# Dropdown for query

In [ ]:
import numpy as np

In [ ]:
from numpy import random

In [ ]:
nums = random.randint(0,len(wordlist),size=10000)

In [ ]:
options = []
for num in nums:
    word = wordlist[num][0]
    options.append({'label': word,'value': word})

In [ ]:
external_stylesheets = ['styles.css']

server = flask.Flask(__name__)

app = dash.Dash(__name__, server=server, external_stylesheets=external_stylesheets)

app.layout = html.Div([
    html.H1('Twitter 1-grams'),
    html.Div([
        dcc.Dropdown(id='my-dropdown', options=options, value=['@realdonaldtrump','covfefe'], multi=True)]),
    dcc.Graph(id='my-graph')
])

@app.callback(Output('my-graph', 'figure'), [Input('my-dropdown', 'value')])

def update_graph(input_value):
    data=[]
    figure = {'data':data}
    for item in input_value:
        df = load(item)
        data.append({'x':df['time'], 'y':df['counts'],'name':item})
    return figure


if __name__ == '__main__':
    app.run_server()